# Solving FizzBuzz with Keras

## Overview
- This notebook provides an example Keras network to solve FizzBuzz.
- This is a supervised classification problem, so we'll train our model on labeled data.

<img src="https://camo.githubusercontent.com/fedd5d66bea57a430635498de58dc7c6f064f280/68747470733a2f2f64707a6268796262327064636a2e636c6f756466726f6e742e6e65742f63686f6c6c65742f466967757265732f303166696730322e6a7067">

## What is FizzBuzz?
FizzBuzz is a common programming interview problem. Here's the setup.
- Write a program that outputs all the integers from 1 to 100.
- If the number is a multpile of 3, then output the string "Fizz"
- If the number is a multiple of 5, then output the string "Buzz"
- If the number is a multiple of 15, output the string "FizzBuzz"
- For any other number, output that, number itself.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# imports
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.layers import Dense
from keras.models import Model

In [ ]:
# Specify the number of binary digits. 
NUM_DIGITS = 10

In [ ]:
## Setup the training data for 101-1024. 1024 is the highest number countable with 10 binary digits
raw_training_data = np.array(range(101, 2**NUM_DIGITS))

In [ ]:
## This is a numpy array of integers.
raw_training_data[1:20]

In [ ]:
# We need to binary encode our inputs
def binary_encode(i, NUM_DIGITS):
    return np.array([i >> d & 1 for d in range(NUM_DIGITS)])

In [ ]:
encoded_training_data = [binary_encode(i, NUM_DIGITS) for i in raw_training_data]
x_train = np.array(encoded_training_data)

In [ ]:
print(binary_encode(1, NUM_DIGITS))
print(binary_encode(2, NUM_DIGITS))
print(binary_encode(3, NUM_DIGITS))

In [ ]:
# Now we'll need to one hot encode the training data for y
def fizz_buzz_encode(i):
    if i % 15 == 0:
        return np.array([0, 0, 0, 1]) # encoding for "fizzbuzz"
    elif i % 5 == 0: 
        return np.array([0, 0, 1, 0]) # encoding for "buzz"
    elif i % 3  == 0: 
        return np.array([0, 1, 0, 0]) # encoding for "fizz"
    else:
        return np.array([1, 0, 0, 0]) # encoding for the number output

In [ ]:
# y_train is the encoded output. 
# This is our "labeled data" for supervised learning
y_train = np.array([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])

In [ ]:
# Now let's build our model, add layers, compile, and fit it!
model = Sequential()

In [ ]:
## Add the model's layers. 
model.add(Dense(1000, input_dim=NUM_DIGITS, activation="relu"))
model.add(Dense(1000, activation="relu"))
model.add(Dense(4, activation="softmax"))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=["accuracy"])

In [ ]:
model.fit(x_train, y_train, nb_epoch=100, batch_size=128)

In [ ]:
# fizzbuzz to binary
def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]


In [ ]:
# Setup x_test  fizzbuzz for prime numbers from 1 to 100
numbers = np.arange(1, 101)
x_test = np.transpose(binary_encode(numbers, NUM_DIGITS))
y_test = model.predict_classes(x_test)

In [ ]:
# Setup predicted output
predictions = np.vectorize(fizz_buzz)(numbers, y_test)
print (predictions)

In [ ]:
# correct answers for fizzbuzz on 1-100, these are our actual values
answer = np.array([])
for i in numbers:
    if i % 15 == 0: 
        answer = np.append(answer, "fizzbuzz")
    elif i % 5 == 0: 
        answer = np.append(answer, "buzz")
    elif i % 3 == 0: 
        answer = np.append(answer, "fizz")
    else: answer = np.append(answer, str(i))
print (answer)

In [ ]:
# Let's evaluate the model's predictions
evaluate = np.array(answer == predictions)
print (np.count_nonzero(evaluate == True) / 100)

In [ ]:
answer == predictions

# Exercise
1. Reassign the NUM_DIGITS to be 5 and re-run the notebook. What do you notice? Take note of the accuracy number.
2. Reassign the NUM_DIGITS to be 11 and re-run the notebook. What do you notice? Take note of the accuracy number. 
3. Now try commenting out the last two model layers so there's only a single layer. Re-run the model to test for accuracy.
4. Un-comment the last two model layers and set all their their activation parameters to "sigmoid" and re-run.
5. Explore https://keras.io/activations/ to see what other activation functions are available. Experiment with some and see how they perform. This is an example of hyperparameter tuning.